In [3]:
#افزودن کتابخونه
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, silhouette_score
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

In [46]:
columns = [
    'تاریخ', 'شیفت', 'نوع لوله', 'قطر (mm)', 'ضخامت (mm)', 'متراژ تولید (m)', 
    'وزن (kg)', '% مواد بازیافتی', 'گرید مواد', 'دمای Zone1 (°C)', 
    'دمای Zone2 (°C)', 'دمای Zone3 (°C)', 'دمای هد (°C)', 'فشار اکسترودر (bar)', 
    'سرعت مارپیچ (rpm)', 'سرعت کشنده (m/min)', 'دمای سالن (°C)', 'رطوبت سالن (%)', 
    'ضایعات (kg)', 'علت ضایعات', 'تست QC'
]

df = pd.read_csv('./data_pipes.csv', names=columns, header=0)

# تبدیل ستون‌های عددی به نوع عددی
numeric_cols = [
    'قطر (mm)', 'ضخامت (mm)', 'متراژ تولید (m)', 'وزن (kg)', 
    '% مواد بازیافتی', 'دمای Zone1 (°C)', 'دمای Zone2 (°C)', 
    'دمای Zone3 (°C)', 'دمای هد (°C)', 'فشار اکسترودر (bar)', 
    'سرعت مارپیچ (rpm)', 'سرعت کشنده (m/min)', 'دمای سالن (°C)', 
    'رطوبت سالن (%)', 'ضایعات (kg)'
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# بررسی NaN و حذف ردیف‌های ناقص
print("Missing values per column:\n", df.isnull().sum())
df.dropna(inplace=True)

# تبدیل ستون‌های متنی به کد عددی با LabelEncoder (برای مدل‌سازی)
categorical_cols = ['شیفت', 'نوع لوله', 'گرید مواد', 'علت ضایعات', 'تست QC']
le_dict = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    le_dict[col] = le  # ذخیره برای بازگرداندن نام‌ها اگر لازم شد

print("\nFinal data shape after cleaning:", df.shape)
print(df.head())


Missing values per column:
 تاریخ                  0
شیفت                   0
نوع لوله               0
قطر (mm)               0
ضخامت (mm)             0
متراژ تولید (m)        0
وزن (kg)               0
% مواد بازیافتی        0
گرید مواد              0
دمای Zone1 (°C)        0
دمای Zone2 (°C)        0
دمای Zone3 (°C)        0
دمای هد (°C)           0
فشار اکسترودر (bar)    0
سرعت مارپیچ (rpm)      0
سرعت کشنده (m/min)     0
دمای سالن (°C)         0
رطوبت سالن (%)         0
ضایعات (kg)            0
علت ضایعات             0
تست QC                 0
dtype: int64

Final data shape after cleaning: (42, 21)
        تاریخ  شیفت  نوع لوله  قطر (mm)  ضخامت (mm)  متراژ تولید (m)  \
0  2025-01-09     1         1       110          10             3833   
1  2025-01-10     2         1       110           6             1440   
2  2025-01-11     1         1        32           3             4343   
3  2025-01-12     1         0       110           2             1164   
4  2025-01-13     1         1  

In [21]:
#جدا سازی قبل ترین
features = df.drop(columns=['تاریخ', 'ضایعات (kg)', 'علت ضایعات', 'تست QC'])
target = df['ضایعات (kg)']
categorical_cols = ['شیفت', 'نوع لوله', 'گرید مواد']
numerical_cols = features.select_dtypes(include=['int64', 'float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

In [50]:
# تقسیم داده‌ها
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [51]:
# شناسایی ستون‌ها
categorical_cols = ['شیفت', 'نوع لوله', 'گرید مواد']
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [53]:
# مدل
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'SVR': SVR(),
    'GradientBoosting': GradientBoostingRegressor(random_state=42)
}

param_grids = {
    'LinearRegression': {},
    'Ridge': {'model__alpha': [0.1, 1.0, 10.0]},
    'Lasso': {'model__alpha': [0.1, 1.0, 10.0]},
    'RandomForest': {'model__n_estimators': [50, 100], 'model__max_depth': [None, 10, 20]},
    'SVR': {'model__C': [0.1, 1, 10], 'model__kernel': ['linear', 'rbf']},
    'GradientBoosting': {'model__n_estimators': [50, 100], 'model__learning_rate': [0.01, 0.1], 'model__max_depth': [3, 5]}
}

In [54]:
#نتیجه 
s = {}

for name, model in models.items():
    pipeline = Pipeline([('preprocessor', preprocessor), ('model', model)])
    grid = GridSearchCV(pipeline, param_grids[name], cv=5, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    mse_val = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse_val)
    
    s[name] = {
        'Best Params': grid.best_params_,
        'R2': round(r2, 4),
        'MSE': round(mse_val, 2),
        'MAE': round(mae, 2),
        'RMSE': round(rmse, 2)
    }

# جدول نتایج مرتب
s_df = pd.DataFrame(s).T.sort_values(by='R2', ascending=False)
print(s_df)

                                                        Best Params      R2  \
SVR                      {'model__C': 1, 'model__kernel': 'linear'} -0.1103   
GradientBoosting  {'model__learning_rate': 0.01, 'model__max_dep... -0.1927   
RandomForest      {'model__max_depth': 10, 'model__n_estimators'... -0.3271   
Lasso                                        {'model__alpha': 10.0}  -0.549   
Ridge                                        {'model__alpha': 10.0} -1.0272   
LinearRegression                                                 {} -2.5986   

                       MSE     MAE    RMSE  
SVR                5595.59   62.22    74.8  
GradientBoosting   6010.98   63.26   77.53  
RandomForest       6688.01   63.97   81.78  
Lasso               7806.5   72.62   88.35  
Ridge             10216.22   83.34  101.08  
LinearRegression  18135.68  106.75  134.67  
